# Automated ML


In [63]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

In [64]:
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [65]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'puru-auto-experiment'

experiment=Experiment(ws, experiment_name)

In [66]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-132551
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-132551


In [67]:
# create or attach an existing compute cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# set name of CPU cluster
cpu_cluster_name = "puru-compute-new"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using it.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# wait for completion
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
I am using external data source from Kaggle. I have already downloaded the data from Kaggle and uploaded in my github project, so that we can give the data source as url. 
The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. The target (label) is "Outcome" column with values of 1 means has diabetes and 0 means no diabetes. We will be using all independent varialbles from the datasets like: preganicies the patient has had, their BMI, insulin level, age and so on.

In [68]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
# NOTE: update the key to match the dataset name
found = False
key = "diabetes-classification"
description_text = "Diabetes datasets for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/purunep/Capstoneproject/main/project/data/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00,768.00
mean,3.85,120.89,69.11,20.54,79.80,31.99,0.47,33.24,0.35
std,3.37,31.97,19.36,15.95,115.24,7.88,0.33,11.76,0.48
min,0.00,0.00,0.00,0.00,0.00,0.00,0.08,21.00,0.00
25%,1.00,99.00,62.00,0.00,0.00,27.30,0.24,24.00,0.00
50%,3.00,117.00,72.00,23.00,30.50,32.00,0.37,29.00,0.00
75%,6.00,140.25,80.00,32.00,127.25,36.60,0.63,41.00,1.00
max,17.00,199.00,122.00,99.00,846.00,67.10,2.42,81.00,1.00


In [69]:
# prepare dataset for model training
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Clean dataset using the clean_data function
x, y = clean_data(dataset)

# split dataset into train and test sets
(x_train, x_test, y_train, y_test) = train_test_split(x, y, test_size= 0.2, random_state = 0)
label = 'Outcome'

# merge the output x and y dataframes into a single table for AutoML experiment
train_data_df = pd.concat([x_train, y_train], axis=1)

train_data_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
603,7,150,78,29,126,35.20,0.69,54,1
118,4,97,60,23,0,28.20,0.44,22,0
247,0,165,90,33,680,52.30,0.43,23,0
157,1,109,56,21,135,25.20,0.83,23,0
468,8,120,0,0,0,30.00,0.18,38,1


In [70]:
from azureml.data.dataset_factory import TabularDatasetFactory
# save training data in tabular format to allow for remote run
if not os.path.isdir('data'):  # create data folder if it does not exist
    os.mkdir('data')
    
if not os.path.exists('project_folder'):  # create project folder if it does not exist
    os.makedirs('project_folder')

# Save the train data to a csv file to be uploaded to the datastore
pd.DataFrame(train_data_df).to_csv("data/train_data.csv", index=False)

# Upload the training data as a tabular dataset for access during training on remote compute
# upload to data store
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='automlclassifier', overwrite=True, show_progress=True)

 # access datastore during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('automlclassifier/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

I set enabled early stopping to True, to enable early termination if the score is not improving in the short term. Also set the iteration_timeout_minutes = 5, to set the maximum time in minutes that each iteration can run for before it terminates.Set max_concurrent_iterations = 4, to specify the maximum number of iterations that would be executed in parallel. Set max_cores_per_iteration = -1, to specify the maximum number of threads to use for a given training iteration. -1 means to use all the possible cores per iteration per child-run and set featurization = auto, to specify wherether featurization should be done automically or not, auto is ued to do it automatically

In [71]:
project_folder = './puruautoml'
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,    
    "featurization": 'auto'
}

# define automl autconfig parameters
automl_config = AutoMLConfig(
    experiment_timeout_minutes= 30,
    task= 'classification',
    primary_metric='accuracy',
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster, # included to allow for remote compute
    training_data= train_data,
    label_column_name= label,
    path = project_folder,
    n_cross_validations=3,
    debug_log = "automl_errors.log",    
    **automl_settings)

In [72]:
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

# wait for completion
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

*******************************************

## Best Model



In [13]:
# Retrieve the best automl run model
best_automl_run, fitted_automl_model = remote_run.get_output()
print(best_automl_run)

# get best model and display properties
model_name = best_automl_run.properties['model_name']
print("Best Model:", model_name)

# display all the properties of the best model
#best_automl_run #.properties
best_automl_run.get_properties()

Run(Experiment: puru-auto-experiment,
Id: AutoML_2a8d0b39-7d5a-4b61-93b2-fc1cfcd64004_36,
Type: azureml.scriptrun,
Status: Completed)
Best Model: AutoML2a8d0b39736


{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'puru-auto-experiment\',\'compute_target\':\'puru-compute-new\',\'subscription_id\':\'61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_2a8d0b39-7d5a-4b61-93b2-fc1cfcd64004_36","experiment_name":"puru-auto-experiment","workspace_name":"quick-starts-ws-132551","subscription_id":"61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30","resource_group_name":"aml-quickstarts-132551"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '36',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azurem

In [14]:
import joblib
joblib.dump(fitted_automl_model, 'automlmodel.pkl')

['automlmodel.pkl']

In [32]:
# register best autoML model for future deployment
from azureml.core.model import Model
model= Model.register(workspace = ws, model_name='automlmodel', model_path='automlmodel.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model automlmodel
automlmodel	automlmodel:2	2


In [33]:
# Save the best model
os.makedirs('outputs', exist_ok=True)
best_automl_run.download_file('outputs/model.pkl', 'outputs/automlmodel.pkl')
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py', 'outputs/score_aml.py')
best_automl_run.download_file('automl_driver.py', 'outputs/automl_driver.py')

In [34]:
import pickle
file = open("outputs/automlmodel.pkl",'rb')
aml_model = pickle.load(file)
file.close()

In [37]:
from train import clean_data
from sklearn.metrics import accuracy_score
import pandas as pd


factory = TabularDatasetFactory()
test_data_path = "https://raw.githubusercontent.com/purunep/Capstoneproject/main/project/data/diabetes.csv"
test_ds = pd.read_csv(test_data_path)
y_test = test_ds[['Outcome']]
print(accuracy_score(aml_model.predict(test_ds.drop(columns=['Outcome'])), y_test))

0.8619791666666666


In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, aml_model.predict(test_ds.drop(columns=['Outcome']))))

              precision    recall  f1-score   support

           0       0.87      0.92      0.90       500
           1       0.84      0.75      0.79       268

    accuracy                           0.86       768
   macro avg       0.86      0.84      0.84       768
weighted avg       0.86      0.86      0.86       768



In [39]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="automlmodel.pkl",
                        model_name = "automl_puru")

Registering model automl_puru


In [40]:

print(model.name)

automl_puru


## Model Deployment

In [16]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

In [57]:
%%writefile autoscore.py
import json
import numpy as np
import os
import pickle
import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'automlmodel.pkl')
    model = joblib.load(model_path)
    

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

Overwriting score.py


In [55]:
%%writefile env.yml
channels:
- anaconda
- conda-forge
dependencies:
- pip
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-telemetry==1.19.0
  - azureml-train-restclients-hyperdrive==1.19.0
  - azureml-train-core==1.19.0
  - azureml-widgets==1.19.0
  - azureml-pipeline-core==1.19.0
  - azureml-pipeline-steps==1.19.0
  - azureml-opendatasets==1.19.0
  - azureml-automl-core==1.19.0
  - azureml-automl-runtime==1.19.0
  - azureml-train-automl-client==1.19.0
  - azureml-train-automl-runtime==1.19.0
  - azureml-train-automl==1.19.0
  - azureml-train==1.19.0
  - azureml-sdk==1.19.0
  - azureml-interpret==1.19.0
  - azureml-tensorboard==1.19.0
  - azureml-mlflow==1.19.0
  - mlflow
  - sklearn-pandas
  - scikit-learn
- pandas
- numpy
- tqdm
- py-xgboost
- joblib
- xgboost
- scikit-learn
- matplotlib
name: azureml_b1836710baabf05d37337a101fa8d6d1

Writing env.yml


In [59]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = Environment.from_conda_specification(name='puruenv',file_path = 'env.yml')
inference_config = InferenceConfig(entry_script='./autoscore.py',
                                    environment=env)

In [61]:
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1,enable_app_insights = True, auth_enabled=True)
service = Model.deploy(ws, "puruautoservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [62]:
servicelogs = service.get_logs()
print(servicelogs)

2020-12-30T23:50:39,532005000+00:00 - gunicorn/run 
2020-12-30T23:50:39,531502100+00:00 - iot-server/run 
2020-12-30T23:50:39,533414800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-30T23:50:39,545084000+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_afca42347aeb1ef9abd9ee384fe31fec/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
print(service.state)
print("scoring URI: " + service.scoring_uri)

print("Swagger URI: " + service.swagger_uri)

print("Authetication Key: " + service.get_keys())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import json

input_payload = json.dumps({
    'data': dataset_x[0:2].tolist(),
    'method': 'predict'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
import json


payload = json.dumps({
    'data':
           [
               {
                   'Pregnancies': 6,
                   'Glucose': 148,
                   'BloodPressure': 72,
                   'SkinThickness': 35,
                   'Insulin': 0,
                   'BMI': 33.6,
                   'DiabetesPedigreeFunction': 0.627,
                   'Age': 50

               },
               {
                   'Pregnancies': 1,
                   'Glucose': 85,
                   'BloodPressure': 66,
                   'SkinThickness': 29,
                   'Insulin': 0,
                   'BMI': 26.6,
                   'DiabetesPedigreeFunction': 0.351,
                   'Age': 31,  
               }
           ],
    'method': 'predict_proba'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(payload)


In [ ]:
import requests
import json
scoringuri = 'http://b2a353bc-5ca6-4fef-ab94-eaca1bbce5c6.southcentralus.azurecontainer.io/score'#3service.scoring_uri
key = 'ZujmvCwycbj01oPTcIAa1KEHjCPXnlyC'#primary
data= { "data":
       [
           {
               'Pregnancies': 6,
               'Glucose': 148,
               'BloodPressure': 72,
               'SkinThickness': 35,
               'Insulin': 0,
               'BMI': 33.6,
               'DiabetesPedigreeFunction': 0.627,
               'Age': 50
               
           },
           {
               'Pregnancies': 1,
               'Glucose': 85,
               'BloodPressure': 66,
               'SkinThickness': 29,
               'Insulin': 0,
               'BMI': 26.6,
               'DiabetesPedigreeFunction': 0.351,
               'Age': 31,  
           }
       ]
    }
input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

response = requests.post(scoringuri, input_data, headers = headers)

In [ ]:
service.delete()